In [25]:
from pprint import pprint
from gdc.data_access import *
from gdc.estimation.consumption import *

### Pooled effect model

In [26]:
p_mean_model = PooledMDHUncorrelatedErrors(
    y=df_load_simulated_normalized,
    temp=df_temp_simulated_normalized
)

beta, means = p_mean_model.fit()

In [27]:
pprint(p_mean_model.summary(beta, means, label="pooled_means"))

{'beta_CDD': 0.003547864966094494,
 'beta_HDD': 0.025797156617045403,
 'label': 'pooled_means',
 'static_fit': {'r2': 0.09677087276703245, 'rmse': 0.8561899656671869}}


### Fixed effect model

In [35]:
i_mean_model = IndividualMDHUncorrelatedErrors(
    y=df_load_simulated_normalized,
    temp=df_temp_simulated_normalized
)

beta, means = i_mean_model.fit()

In [36]:
pprint(i_mean_model.summary(beta, means, label="iMDH_means"))

{'beta_CDD': 0.002452349755913019,
 'beta_HDD': 0.030275173485279083,
 'label': 'iMDH_means',
 'static_fit': {'r2': 0.533043806097159, 'rmse': 0.6156150617453132}}


In [47]:
error_ar1_24 = ARErrorModel(
    i_mean_model.static_resids(beta, means), lags=[1, 24])
error_ar1_24_res = error_ar1_24.fit()
summary = i_mean_model.summary(
            beta, means, error_model_=error_ar1_24_res,
            label="IndividualMDH with AR(1, 24) errors")
pprint(summary)

{'beta_CDD': 0.002452349755913019,
 'beta_HDD': 0.030275173485279083,
 'dynamic_fit': {'coeffs': [0.25199447309015743, 0.44859790182903275],
                 'lags': [1, 24],
                 'r2': 0.6709014815856378,
                 'rmse': 0.516813833857603},
 'label': 'IndividualMDH with AR(1, 24) errors',
 'static_fit': {'r2': 0.533043806097159, 'rmse': 0.6156150617453132}}


In [40]:
def adjusted_r2(r2, N, P):
    return 1- (1-r2)*(N-1)/(N-P-1)
    

In [44]:
T, S = df_load_simulated_normalized.shape
N = T * S

In [45]:
P_HMD = 24 + 7 + 12
P_iHMD = P_HMD * S
P_iHMD_AR = P_iHMD + 2
P_HMD, P_iHMD, P_iHMD_AR, N

(43, 430000, 430002, 87120000)

In [46]:
adjusted_r2(0.09677087276703245, N, P_HMD)

0.09677042695812421

In [48]:
adjusted_r2(0.533043806097159, N, P_iHMD)

0.5307276085461794

In [49]:
adjusted_r2(0.6709014815856378, N, P_iHMD_AR)

0.6692690784709484